**Вариант 1**

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

1. Наименование вакансии.
2. Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
3. Ссылку на саму вакансию.
4. Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [230]:
import pandas as pd
import requests
import json

from lxml import html
from pprint import pprint
from bs4 import BeautifulSoup as bs

In [231]:
def one_vacancy_parser(vacancy_description):
    vacancy_data = {}
    vacancy_data['website'] = 'hh.ru'
    
    v_t = vacancy_description.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
    if not v_t:
        vacancy_data['title'] = None
    else:
        vacancy_data['title'] = v_t.text
    
    v_h = vacancy_description.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
    if not v_h:
        vacancy_data['title'] = None
    else:
        vacancy_data['href'] = v_h['href']
        
    
    v_a = vacancy_description.find('div', {'data-qa':'vacancy-serp__vacancy-address'})
    if not v_a:
        vacancy_data['adress'] = None
    else:
        vacancy_data['adress'] = v_a.text    
    
    vacancy_salary = vacancy_description.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
    if not vacancy_salary:
        vacancy_data['min_salary'] = None
        vacancy_data['max_salary'] = None
        vacancy_data['currency'] = None
    else:
        v_s = vacancy_salary.text.replace('\u202f', '').split(' ')
        if v_s[0] == 'до':
            vacancy_data['min_salary'] = None
            vacancy_data['max_salary'] = v_s[1]
            vacancy_data['currency'] = v_s[2]
        elif v_s[0] == 'от':
            vacancy_data['min_salary'] = v_s[1]
            vacancy_data['max_salary'] = None
            vacancy_data['currency'] = v_s[2]
        else:
            vacancy_data['min_salary'] = v_s[0]
            vacancy_data['max_salary'] = v_s[2]
            vacancy_data['currency'] = v_s[3]
    return vacancy_data

In [232]:
def vacancy_parser(vacancy, area):
    vacancy_data = []
    params = {'text': vacancy, 'area': area, 'search_field': 'name', \
              'items_on_page': '50', 'page': ''} #параметры запроса, берем по всей России
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36'} # задаем чтобы нас роботом не воспринимали

    url = 'https://hh.ru/search/vacancy' # адрес сайта для парсинга(в данном случае ищем на hh.ru вакансию)
    req = requests.get(url, params=params, headers=headers)

    if req.status_code != requests.codes.ok: # проверяем на код 200
            raise Exception(f"http code == {r.status_code}")
    else:   
        html_page = bs(req.text, 'html.parser') # сохраняем код html

    page_html_counter = html_page.find('div', {'data-qa': 'pager-block'})
    if not page_html_counter:
        last_page = 1
    else:
        for page_elem in page_html_counter.find_all('a', {'data-qa': 'pager-page'}):
            last_page = int(page_elem.find('span').text) #Определяем номер последней страницы в ответе на запрос

    for page in range(0, last_page): 
        params['page'] = page
        req = requests.get(url, params=params, headers=headers) #пробегаемся по всем страницам ответа на запрос

        if req.status_code == requests.codes.ok:
            html_page = bs(req.text,'html.parser') #сохроаняем html каждой страницы

            vacancy_array = html_page.find('div', {'data-qa': 'vacancy-serp__results'})\
                .find_all('div', {'class': 'vacancy-serp-item'}) #все вакансии на текущей странице
            for vacancy_description in vacancy_array:
                vacancy_data.append(one_vacancy_parser(vacancy_description)) #пробегаемся по вакансиям и разбираем каждую
            
    return vacancy_data #возвращаем в итоге список словарей с вакансиями

In [235]:
df = pd.DataFrame(vacancy_parser('python', '1'))
df.head(6)

,website,title,href,adress,min_salary,max_salary,currency
0,hh.ru,Backend Python Developer,https://hh.ru/vacancy/54959111?from=vacancy_se...,"Москва, Деловой центр и еще 2",80000,80000,руб.
1,hh.ru,Python разработчик,https://hh.ru/vacancy/54939886?from=vacancy_se...,Москва,None,450000,руб.
2,hh.ru,Разработчик Solidity/Разработчик Rust/Разработ...,https://hh.ru/vacancy/48457284?from=vacancy_se...,Москва,600000,None,руб.
3,hh.ru,Python Developer,https://hh.ru/vacancy/54929214?from=vacancy_se...,Москва,250000,350000,руб.
4,hh.ru,Python разработчик (Django),https://hh.ru/vacancy/54962606?from=vacancy_se...,Москва,60000,None,руб.
5,hh.ru,Младший разработчик Python,https://hh.ru/vacancy/54563092?from=vacancy_se...,"Москва, Калужская",None,None,None


In [240]:
json_hh = json.dumps(vacancy_parser('python', '1'))

In [239]:
with open(f'Lesson2_hh_ru.json', 'w') as f:
        json.dump(json_hh, f)